# Create and deploy a custom model

## Import libraries

Here we are importing necessary libraries and modules. The code is initializing a Sagemaker session and a Bedrock client to interact with Amazon Sagemaker and Bedrock services via APIs respectively.

In [ ]:
import boto3
from sagemaker import Session
import uuid  # Import the 'uuid' module for generating a unique identifier

# Create a session using the provided AWS SDK sessions
session = Session(boto_session=boto3.session.Session(),
                sagemaker_client=boto3.client('sagemaker'),
                sagemaker_runtime_client=boto3.client('runtime.sagemaker'))

# Initialize Bedrock client
bedrock = boto3.client(service_name='bedrock')

## Set parameters for the training job

Retrieve the Bedrock's IAM Execution Role and its ARN. 


>If you are an AWS Workshop participant, the role was automatically created for you during the environment's provisioning.

>If you are using your own AWS account, please make sure to execute instructions in notebook `00-setup-for-self-paced-users.ipynb`

Here we are initializing an IAM client. Information about the Bedrock IAM role is retrieved and the role ARN is saved to create a customization job on Bedrock.

In [ ]:
client = boto3.client(service_name='iam')
response = client.get_role(
    RoleName='Bedrock-Exec-Role-For-Workshop'
)

roleArn = response["Role"]["Arn"]
#pprint.pp(role_arn)

print(roleArn)

We are configuring the necessary parameters like setting the base model id, training data S3 locations etc. to initialize a Bedrock model customization job that will perform continued pre-training of the Titan model.

In [ ]:
# Select the foundation model you want to customize
base_model_id = "amazon.titan-text-express-v1"

job_prefix = "customTitan"

# Generate a unique identifier for the job and custom model name
job_uuid = str(uuid.uuid4())[:8]  # Extracting the first 8 characters for brevity
jobName = f"{job_prefix}-{job_uuid}"
customModelName = f"{job_prefix}-{job_uuid}"

# Retrieve the default bucket name from the session
bucket = session.default_bucket()

s3_train_data = f"s3://{bucket}/PreProcessed/capmarkets-jargon.jsonl"


## Trigger the bedrock training job

We create a continued pre-training job using the Bedrock client specifying the required configuration. Once created, the job identifier is printed. This identifier can be used to track the status and results of the job.

In [ ]:
jobIdentifier = bedrock.create_model_customization_job(
    customizationType="CONTINUED_PRE_TRAINING",
    jobName=jobName,
    customModelName=customModelName,
    roleArn=roleArn,
    baseModelIdentifier=base_model_id,
    hyperParameters = {
        "epochCount": "5",
        "batchSize": "1",
        "learningRate": "0.00001",
    },
    trainingDataConfig={"s3Uri": s3_train_data},
    outputDataConfig={"s3Uri": f"s3://{bucket}/CustomModel/"},
)

## Monitor the job till the status is shown as "Completed"

In [ ]:
pretrain_job = bedrock.get_model_customization_job(jobIdentifier=jobIdentifier['jobArn'])
print(pretrain_job['status'])

## Create provisioned no-commit throughput for the custom model

 (Only run the following once the status of the above job is shown as "Completed")

This code is configuring the provisioned inference capacity for the custom model resulting from the continued pre-training job, so it can be deployed as a Bedrock managed endpoint.

In [ ]:
customModelId=pretrain_job['outputModelArn']

provisionedModelName = f"{job_prefix}-provisioned-{job_uuid}"

# Create the provisioned capacity without passing any commitment option
provisionedModelArn = bedrock.create_provisioned_model_throughput(
    modelUnits=1,
    provisionedModelName=provisionedModelName, 
    modelId=customModelId
   )['provisionedModelArn']

## Check the provisoned capacity creation status

In [ ]:
# Get Provisioned model status untill it's completed
provisionedModelStatus = bedrock.get_provisioned_model_throughput(provisionedModelId=provisionedModelArn)
print (provisionedModelStatus['status'])

## The following values will be referred in the next two notebooks

In [ ]:
%store provisionedModelArn
%store customModelId
%store base_model_id